<a href="https://colab.research.google.com/github/jiw3026/RNDix_Tokenizer/blob/main/RNDix_Baseline_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import glob
import json
import random
import torch
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Mecab

In [2]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 26.6 MB/s eta 0:00:00


In [4]:
# Drive 마운트 및 데이터 로드
def load_kowiki(file_path, num_sentences=None):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.readlines()
    data = [line.strip() for line in data if line.strip()]
    return data[:num_sentences] if num_sentences else data

kowiki_train = load_kowiki("/content/kowikitext_20200920.train")
kowiki_dev = load_kowiki("/content/kowikitext_20200920.dev")
kowiki_test = load_kowiki("/content/kowikitext_20200920.dev")


In [8]:
# 2022_3q 데이터 로드 및 Train/Test Split
def load_texts_from_dir(directory):
    files = glob.glob(os.path.join(directory, "*.txt"))
    texts = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            texts.extend(f.readlines())
    return texts

q3_texts = load_texts_from_dir("/content/drive/MyDrive/Tokenizer")

# 80:20 비율로 train/test split
random.shuffle(q3_texts)
train_texts = q3_texts[:int(len(q3_texts) * 0.8)]
test_texts = q3_texts[int(len(q3_texts) * 0.2):]

In [9]:
train_texts

['음바페 홀란드에 묻힌다.\n',
 '출마 자격 안되는 사람의 출마를 논의 한다는 것 자체가 특혜다. 민주당은 정신 차려라.\n',
 '아무리 윤재앙 거려도 지금 전국민이 문재인 똥치우라고 윤석열 뽑은건 변하지 않는다\n',
 '아......부모가슴이...ㅠㅠ\n',
 '울고불고~ 해도 버스는 지나갔다. 그러게 학교다닐때 공부는 안해도 좋다만,,, 맘씨를 곱게써야되는거야. 외모믿고 까부는것들,,,그게바로 인과응보지.\n',
 '너무실망이다. 이재명의 음주운전은 비난대상이나 내편이면 강행...\n',
 '느낌이 아니라 실제로 그럼\n',
 '당신과 같은 대한민국 출신이지\n',
 '\n',
 '\n',
 '\n',
 '천만뷰 기록의 가수네요\n',
 '대깨들의 발정난 충신들이냐?...\n',
 '\n',
 '이시대에 정치평론가라 자처하는 인간들은 너 자신을알고 남을 평가하라고 말하고싶다. 가슴은 엉망진탕 부패하면서 입으로만 잘난체 남탓만하는 종자들은 스스로 너자신을 돌아보고 남탓을 하라. 개나소나 자칭 정치평론가라 하는 종자들을 증오한다.\n',
 '\n',
 '\n',
 '규제고 나발이고 부동산은 끝모를 추락만 남았다 네이버부동산 가격은 호가일뿐 거래가 없어요 거래가\n',
 '\n',
 '기다렸다 7월13일 금리 오르면 가입하라~\n',
 '\n',
 '간첩 한마리등장했다 아이디조사해서 빨리 북한으로 추방하라\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '애미애비다\n',
 '\n',
 '니 가족과 동료도 좌빨이나 빨갱이냐\n',
 '한동훈장관이 아무리 뛰어나도 뿌리를 내려야할 토양이 이렇게 혼란스러우면 큰나무 되기가 힘들수 밖에 앖다 제발 국힘 ㅡㅡㅡ 쫌\n',
 '돈은 부족해도 법카가 있잖아.\n',
 '맞는 말만 하네\n',
 '와 보수꼴통 극우유튜버 다 모이는거냐???????????ㅋㅋㅋㅋㅋㅋ 손가락 자르고싶네 ㄹㅇ\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '지들도 맨날 노가다나 일용직 하면서

In [10]:

## 2. CharEncoder: 어휘 사전 만들기
class CharEncoder:
    def __init__(self):
        self.char2idx = {'<PAD>': 0, '<UNK>': 1}
        # 한글 음절 추가 (가~힣 범위로 unknown 최소화)
        for char in [chr(i) for i in range(ord('가'), ord('힣') + 1)]:
            self.add_character(char)

    def add_character(self, char):
        if char not in self.char2idx:
            self.char2idx[char] = len(self.char2idx)

    def encode(self, text):
        return [self.char2idx.get(char, self.char2idx['<UNK>']) for char in text]

    def save_vocab(self, path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.char2idx, f, ensure_ascii=False, indent=4)

    def load_vocab(self, path):
        with open(path, 'r', encoding='utf-8') as f:
            self.char2idx = json.load(f)

vocab = CharEncoder()

# Train 텍스트에서 문자 추가
for text in train_texts + kowiki_train:
    for char in text:
        vocab.add_character(char)

vocab.save_vocab("/content/drive/MyDrive/char_vocab.json")

In [11]:
## 3. CharDecoder: 인코딩된 숫자열을 문자열로 변환
class CharDecoder:
    def __init__(self, char2idx):
        self.idx2char = {idx: char for char, idx in char2idx.items()}

    def decode(self, indices):
        return ''.join([self.idx2char.get(idx, '') for idx in indices])

decoder = CharDecoder(vocab.char2idx)

In [12]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.


In [13]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [14]:
# Mecab 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,000 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,257 kB in 2s (1,866 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy I

In [15]:
!apt-get update
!apt-get install g++ openjdk-8-jdk

# Install Konlpy and JPype
!pip3 install konlpy JPype1-py3

# Download the installation script and install mecab-ko
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 1,760 kB in 1s (1,191 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https:/

In [16]:
# 시스템 패키지 업데이트
!apt-get install g++ openjdk-8-jdk python3-dev

# Mecab & 관련 패키지 설치
!apt-get install mecab libmecab-dev mecab-ko mecab-ko-dic

# Python에서 mecab-ko를 사용하기 위해 필요한 패키지 설치
!pip install konlpy mecab-python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
python3-dev is already the newest version (3.10.6-1~22.04.1).
openjdk-8-jdk is already the newest version (8u442-b06~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package mecab-ko
E: Unable to locate package mecab-ko-dic


In [17]:
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.morphs("테스트 문장입니다."))

['테스트', '문장', '입니다', '.']


In [9]:
## 4. BIOTaggingDataset: B/I 태그 및 데이터셋 구축

def generate_bio_tags(texts, tokenizer, max_length=128):
    labels, masks = [], []
    for text in texts:
        tokens = tokenizer(text)
        label = [0] * len(text)

        idx = 0
        for token in tokens:
            if idx < len(text):
                label[idx] = 1
                idx += len(token)

        # 길이 맞추기
        if len(label) < max_length:
            label += [0] * (max_length - len(label))
            mask = [1] * len(label) + [0] * (max_length - len(label))
        else:
            label = label[:max_length]
            mask = [1] * max_length

        labels.append(label)
        masks.append(mask)

    return torch.tensor(labels), torch.tensor(masks)

mecab = Mecab()
train_labels, train_masks = generate_bio_tags(train_texts, mecab.morphs)

KeyboardInterrupt: 

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Mecab

# ✅ 배치 처리를 위한 Dataset 클래스 정의
class BIOTaggingDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        # Mecab으로 형태소 분석 → B/I 태그 생성
        tokens = self.tokenizer(text)

        label = [0] * len(text)
        idx = 0
        for token in tokens:
            if idx < len(text):
                label[idx] = 1  # 토큰의 시작 위치는 B(1)로 지정
                idx += len(token)

        # 길이 맞추기 (padding 또는 자르기)
        if len(label) < self.max_length:
            mask = [1] * len(label) + [0] * (self.max_length - len(label))
            label += [0] * (self.max_length - len(label))
        else:
            mask = [1] * self.max_length
            label = label[:self.max_length]

        return torch.tensor(label, dtype=torch.long), torch.tensor(mask, dtype=torch.bool)

# ✅ 병렬 처리를 위한 DataLoader 생성 함수
def create_bio_dataloader(texts, tokenizer, batch_size=32, max_length=128, num_workers=4):
    dataset = BIOTaggingDataset(texts, tokenizer, max_length)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# ✅ Mecab 초기화 및 DataLoader 생성
mecab = Mecab()
batch_size = 64  # 배치 크기 조절로 속도 최적화
num_workers = 8  # CPU 코어 수에 맞춰 설정 (Colab은 2~4 권장)

# BI 태깅 데이터 생성 (예: train_texts를 활용)
train_loader = create_bio_dataloader(train_texts, mecab.morphs, batch_size=batch_size, num_workers=num_workers)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [3]:
!pip install pytorch-crf
from torchcrf import CRF

In [19]:
## 5. Neural Tokenizer - Baseline (BiLSTM + CRF)

import torch.nn as nn
import torch.optim as optim
from torchcrf import CRF

class NeuralTokenizer(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=128, num_labels=2):
        super(NeuralTokenizer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=2, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, x, labels=None, mask=None):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        emissions = self.fc(x)

        if labels is not None:
            loss = -self.crf(emissions, labels, mask=mask, reduction='mean')
            return loss
        else:
            return self.crf.decode(emissions, mask=mask)

model = NeuralTokenizer(vocab_size=len(vocab.char2idx))
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Mecab

# ✅ Device 설정 (GPU가 있으면 GPU 사용, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ 배치 처리를 위한 Dataset 클래스 정의
class BIOTaggingDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        # Mecab으로 형태소 분석 → B/I 태그 생성
        tokens = self.tokenizer(text)

        label = [0] * len(text)
        idx = 0
        for token in tokens:
            if idx < len(text):
                label[idx] = 1  # 토큰의 시작 위치는 B(1)로 지정
                idx += len(token)

        # 길이 맞추기 (padding 또는 자르기)
        if len(label) < self.max_length:
            mask = [1] * len(label) + [0] * (self.max_length - len(label))
            label += [0] * (self.max_length - len(label))
        else:
            mask = [1] * self.max_length
            label = label[:self.max_length]

        return torch.tensor(label, dtype=torch.long), torch.tensor(mask, dtype=torch.bool)

# ✅ 병렬 처리를 위한 DataLoader 생성 함수
def create_bio_dataloader(texts, tokenizer, batch_size=32, max_length=128, num_workers=4):
    dataset = BIOTaggingDataset(texts, tokenizer, max_length)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# ✅ Mecab 초기화 및 DataLoader 생성
mecab = Mecab()
batch_size = 64  # 배치 크기 조절로 속도 최적화
num_workers = 8  # Colab은 2~4 권장

# BI 태깅 데이터 생성 (train_texts를 활용)
train_loader = create_bio_dataloader(train_texts, mecab.morphs, batch_size=batch_size, num_workers=num_workers)

# ✅ 데이터 검증: 샘플 출력
for labels, masks in train_loader:
    print(f"Sample BI Labels Shape: {labels.shape}")
    print(f"Sample BI Labels (First Batch):\n{labels[:2]}")
    print(f"Sample Masks (First Batch):\n{masks[:2]}")
    break

Using device: cpu


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Sample BI Labels Shape: torch.Size([64, 128])
Sample BI Labels (First Batch):
tensor([[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
         0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
         0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
         1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
Sample Masks (First Batch):
tensor([[ True,

In [22]:
## 6. Trainer: 학습 루프
class Trainer:
    def __init__(self, model, optimizer, train_loader):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.train_loader = train_loader

    def train(self, epochs=5):
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for x, y, m in self.train_loader:
                x, y, m = x.to(device), y.to(device), m.to(device)
                m[:, 0] = True

                self.optimizer.zero_grad()
                loss = self.model(x, y, mask=m)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                self.optimizer.step()
                total_loss += loss.item()

            print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

trainer = Trainer(model, optimizer, train_loader)
trainer.train(epochs=5)

ValueError: not enough values to unpack (expected 3, got 2)

In [23]:
### BIOTaggingdataset에서 x(입력 데이터) 누락 문제 해결
class BIOTaggingDataset(Dataset):
    def __init__(self, texts, tokenizer, vocab, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        # 1. 입력 데이터 (x) : 문자 → 정수 변환
        encoded_text = [self.vocab.char2idx.get(char, self.vocab.char2idx["<UNK>"]) for char in text]

        # 2. B/I 라벨 생성
        tokens = self.tokenizer(text)
        label = [0] * len(text)
        idx = 0
        for token in tokens:
            if idx < len(text):
                label[idx] = 1  # B 태그 = 1
                idx += len(token)

        # 3. Padding (길이 맞추기)
        if len(encoded_text) < self.max_length:
            mask = [1] * len(encoded_text) + [0] * (self.max_length - len(encoded_text))
            encoded_text += [self.vocab.char2idx["<PAD>"]] * (self.max_length - len(encoded_text))
            label += [0] * (self.max_length - len(label))
        else:
            mask = [1] * self.max_length
            encoded_text = encoded_text[:self.max_length]
            label = label[:self.max_length]

        return torch.tensor(encoded_text, dtype=torch.long), torch.tensor(label, dtype=torch.long), torch.tensor(mask, dtype=torch.bool)


In [24]:
# DataLoader 생성 함수 (입력, 라벨, 마스크 반환)
def create_bio_dataloader(texts, tokenizer, vocab, batch_size=32, max_length=128, num_workers=4):
    dataset = BIOTaggingDataset(texts, tokenizer, vocab, max_length)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# DataLoader 생성
train_loader = create_bio_dataloader(train_texts, mecab.morphs, vocab, batch_size=64, num_workers=8)


In [25]:
class Trainer:
    def __init__(self, model, optimizer, train_loader):
        self.model = model.to(device)  # GPU/CPU로 모델 이동
        self.optimizer = optimizer
        self.train_loader = train_loader

    def train(self, epochs=5):
        self.model.train()  # 학습 모드로 설정
        for epoch in range(epochs):
            total_loss = 0
            for x, y, m in self.train_loader:  # (입력, 라벨, 마스크)
                # GPU로 이동
                x, y, m = x.to(device), y.to(device), m.to(device)

                # 첫 번째 timestep의 mask는 반드시 True로 설정 (CRF 조건 충족)
                m[:, 0] = True

                self.optimizer.zero_grad()  # 그래디언트 초기화
                loss = self.model(x, y, mask=m)  # 손실 계산
                loss.backward()  # 역전파
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)  # Gradient Clipping
                self.optimizer.step()  # 매개변수 업데이트
                total_loss += loss.item()

            print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Trainer 인스턴스 생성 및 학습 실행
trainer = Trainer(model, optimizer, train_loader)
trainer.train(epochs=5)


KeyboardInterrupt: 